# 요금 한도를 처리하는 방법

OpenAI API를 반복적으로 호출하면 `429: '요청이 너무 많습니다'` 또는 `RateLimitError`라는 오류 메시지가 표시될 수 있습니다. 이러한 오류 메시지는 API의 속도 제한을 초과하여 발생합니다.

이 가이드에서는 전송률 제한 오류를 방지하고 처리하기 위한 팁을 공유합니다.

속도 제한 오류를 피하기 위해 병렬 요청을 스로틀링하는 스크립트 예제를 보려면 [api_request_parallel_processor.py](api_request_parallel_processor.py)를 참조하세요.

## 속도 제한이 존재하는 이유

속도 제한은 API의 일반적인 관행이며, 몇 가지 다른 이유로 인해 설정됩니다.

- 첫째, API의 남용 또는 오용을 방지하는 데 도움이 됩니다. 예를 들어, 악의적인 공격자가 API에 과부하를 일으키거나 서비스 중단을 유발하기 위해 요청을 폭주시킬 수 있습니다. 속도 제한을 설정하면 이러한 종류의 활동을 방지할 수 있습니다.
- 둘째, 속도 제한은 모든 사람이 API에 공정하게 액세스할 수 있도록 보장합니다. 한 사람이나 조직이 과도하게 많은 요청을 하면 다른 모든 사람의 API가 느려질 수 있습니다. OpenAI는 한 사용자가 요청할 수 있는 요청 수를 제한함으로써 모든 사용자가 속도 저하 없이 API를 사용할 수 있도록 보장합니다.
- 마지막으로, 속도 제한은 OpenAI가 인프라의 총 부하를 관리하는 데 도움이 될 수 있습니다. API에 대한 요청이 급격히 증가하면 서버에 부담을 주고 성능 문제를 일으킬 수 있습니다. 속도 제한을 설정하면 OpenAI는 모든 사용자에게 원활하고 일관된 환경을 유지할 수 있습니다.

속도 제한에 도달하는 것은 실망스러울 수 있지만, 속도 제한은 사용자를 위한 API의 안정적인 운영을 보호하기 위해 존재합니다.

기본 요율 한도 ##

2023년 1월부터 기본 이자율 한도는 다음과 같습니다:

<표>
<thead>
  <tr>
    <th></th>
    <th>텍스트 완성 및 엔드포인트 임베딩</th>
    <th>코드 및 엔드포인트 편집</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>무료 평가판 사용자</td>
    <td>
        <ul>
            <li>20 요청/분</li>
            <li>150,000토큰/분</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 요청/분</li>
            <li>150,000토큰/분</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>종량제 사용자(첫 48시간 내)</td>
    <td>
        <ul>
            <li>60 요청/분</li>
            <li>250,000 다빈치 토큰/분(더 저렴한 모델의 경우 비례적으로 더 많음)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 요청/분</li>
            <li>150,000토큰/분</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>종량제 사용자(첫 48시간 이후)</td>
    <td>
        <ul>
            <li>3,000 요청/분</li>
            <li>250,000 다빈치 토큰/분(더 저렴한 모델의 경우 비례적으로 더 많음)</li>
        </ul>
    </td>
    <td>
        <ul>
            <li>20 요청/분</li>
            <li>150,000토큰/분</li>
        </ul>
    </td>
  </tr>
</tbody>
</table>

참고로 1,000개의 토큰은 대략 텍스트 한 페이지에 해당합니다.

### 기타 속도 제한 리소스

다른 리소스에서 OpenAI의 속도 제한에 대해 자세히 알아보세요:

- 가이드: 속도 제한](https://beta.openai.com/docs/guides/rate-limits/overview)
- 도움말 센터: API 사용에 속도 제한이 적용되나요?(https://help.openai.com/en/articles/5955598-is-api-usage-subject-to-any-rate-limits)
- 도움말 센터: 429: '너무 많은 요청' 오류를 해결하려면 어떻게 해야 하나요?](https://help.openai.com/en/articles/5955604-how-can-i-solve-429-too-many-requests-errors)

### 비율 제한 증가 요청하기

조직의 요금 한도를 늘리려면 다음 양식을 작성해 주세요:

- OpenAI 요금 한도 증액 요청 양식](https://forms.gle/56ZrwXXoxAN1yt6i9)


## 속도 제한 오류 예시

API 요청이 너무 빨리 전송되면 속도 제한 오류가 발생합니다. OpenAI Python 라이브러리를 사용하는 경우 다음과 같이 표시됩니다:

```
RateLimitError: 조직 org-{id}의 기본 코덱스에 대한 분당 요청 수에 대한 속도 제한에 도달했습니다. 제한: 20.000000/분. 현재: 24.000000/분. 문제가 계속되거나 증가를 요청하려면 support@openai.com 으로 문의하세요.
```

다음은 속도 제한 오류를 트리거하는 예제 코드입니다.

In [ ]:
import openai  # for making OpenAI API requests

# request a bunch of completions in a loop
for _ in range(100):
    openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": "Hello"}],
        max_tokens=10,
    )


속도 제한 오류를 방지하는 방법 ##

지수 백오프로 재시도하기 ###

비율 제한 오류를 피하는 쉬운 방법 중 하나는 무작위 지수 백오프를 사용하여 요청을 자동으로 재시도하는 것입니다. 지수 백오프를 사용하여 재시도한다는 것은 비율 제한 오류가 발생하면 짧은 절전을 수행한 다음 실패한 요청을 다시 시도하는 것을 의미합니다. 요청이 여전히 실패하면 절전 시간이 길어지고 프로세스가 반복됩니다. 이 과정은 요청이 성공하거나 최대 재시도 횟수에 도달할 때까지 계속됩니다.

이 접근 방식에는 많은 이점이 있습니다:

- 자동 재시도는 충돌이나 데이터 누락 없이 속도 제한 오류에서 복구할 수 있음을 의미합니다.
- 지수 백오프를 사용하면 첫 번째 재시도를 빠르게 시도할 수 있으며, 처음 몇 번의 재시도가 실패할 경우 지연 시간이 길어지는 이점을 누릴 수 있습니다.
- 지연에 무작위 지터를 추가하면 모든 히트에서 동시에 재시도하는 데 도움이 됩니다.

실패한 요청은 분당 한도에 영향을 미치므로 요청을 계속 재전송하면 작동하지 않습니다.

다음은 몇 가지 해결 방법의 예입니다.

#### 예시 #1: 끈기 라이브러리 사용

[Tenacity](https://tenacity.readthedocs.io/en/latest/)는 거의 모든 항목에 재시도 동작을 추가하는 작업을 간소화하기 위해 Python으로 작성된 Apache 2.0 라이선스 범용 재시도 라이브러리입니다.

요청에 기하급수적 백오프를 추가하려면 `tenacity.retry` [데코레이터](https://peps.python.org/pep-0318/)를 사용하면 됩니다. 다음 예제는 `tenacity.wait_random_exponential` 함수를 사용하여 요청에 무작위 지수 백오프를 추가하는 예제입니다.

Tenacity 라이브러리는 타사 도구이며, OpenAI는 안정성이나 보안에 대해 보장하지 않는다는 점에 유의하세요.

In [1]:
import openai  # for OpenAI API calls
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
)  # for exponential backoff


@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completion_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowO391reUW8RGVfFyzBM1uBs4A5 at 0x10d8cae00> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " a little girl dreamed of becoming a model.\n\nNowadays, that dream"
    }
  ],
  "created": 1662793900,
  "id": "cmpl-5oowO391reUW8RGVfFyzBM1uBs4A5",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예제 #2: 백오프 라이브러리 사용

백오프 및 재시도를 위한 함수 데코레이터를 제공하는 또 다른 라이브러리는 [backoff](https://pypi.org/project/backoff/)입니다.

Tenacity와 마찬가지로 백오프 라이브러리는 타사 도구이며, OpenAI는 안정성이나 보안에 대해 보장하지 않습니다.

In [2]:
import backoff  # for exponential backoff
import openai  # for OpenAI API calls


@backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowPhIdUvshEsF1rBhhwE9KFfI3M at 0x111043680> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " two children lived in a poor country village. In the winter, the temperature would"
    }
  ],
  "created": 1662793901,
  "id": "cmpl-5oowPhIdUvshEsF1rBhhwE9KFfI3M",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

#### 예제 3: 수동 백오프 구현

타사 라이브러리를 사용하고 싶지 않다면 자체 백오프 로직을 구현할 수 있습니다.

In [3]:
# imports
import random
import time

import openai

# define a retry decorator
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 1,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
    errors: tuple = (openai.error.RateLimitError,),
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            try:
                return func(*args, **kwargs)

            # Retry on specified errors
            except errors as e:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    raise Exception(
                        f"Maximum number of retries ({max_retries}) exceeded."
                    )

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)

            # Raise exceptions for any errors not specified
            except Exception as e:
                raise e

    return wrapper


@retry_with_exponential_backoff
def completions_with_backoff(**kwargs):
    return openai.Completion.create(**kwargs)


completions_with_backoff(model="text-davinci-002", prompt="Once upon a time,")


<OpenAIObject text_completion id=cmpl-5oowRsCXv3AkUgVJyyo3TQrVq7hIT at 0x111024220> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " a man decided to greatly improve his karma by turning his life around.\n\n"
    }
  ],
  "created": 1662793903,
  "id": "cmpl-5oowRsCXv3AkUgVJyyo3TQrVq7hIT",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}

속도 제한이 주어진 일괄 처리의 처리량을 최대화하는 방법 ##

사용자의 실시간 요청을 처리하는 경우, 백오프 후 재시도는 속도 제한 오류를 방지하면서 지연 시간을 최소화할 수 있는 훌륭한 전략입니다.

그러나 지연 시간보다 처리량이 더 중요한 대량의 배치 데이터를 처리하는 경우, 백오프 및 재시도 외에 몇 가지 다른 방법을 사용할 수 있습니다.

### 요청 사이에 사전적으로 지연 시간 추가하기

속도 제한에 도달했다가 백오프하고, 다시 속도 제한에 도달하고, 다시 백오프하는 일이 반복되면 요청 예산의 상당 부분이 재시도해야 하는 요청에 '낭비'될 수 있습니다. 이렇게 하면 고정된 비율 제한이 주어졌을 때 처리량이 제한됩니다.

여기서 한 가지 잠재적인 해결책은 속도 제한을 계산하고 그 역수와 동일한 지연을 추가하는 것입니다(예: 속도 제한이 분당 요청 20건인 경우 각 요청에 3~6초의 지연을 추가). 이렇게 하면 속도 제한 한도에 도달하여 낭비되는 요청을 발생시키지 않고 한도 근처에서 운영할 수 있습니다.

#### 요청에 지연을 추가하는 예

In [4]:
# imports
import time
import openai

# Define a function that adds a delay to a Completion API call
def delayed_completion(delay_in_seconds: float = 1, **kwargs):
    """Delay a completion by a specified amount of time."""

    # Sleep for the delay
    time.sleep(delay_in_seconds)

    # Call the Completion API and return the result
    return openai.Completion.create(**kwargs)


# Calculate the delay based on your rate limit
rate_limit_per_minute = 20
delay = 60.0 / rate_limit_per_minute

delayed_completion(
    delay_in_seconds=delay,
    model="text-davinci-002",
    prompt="Once upon a time,"
)


<OpenAIObject text_completion id=cmpl-5oowVVZnAzdCPtUJ0rifeamtLcZRp at 0x11b2c7680> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " there was an idyllic little farm that sat by a babbling brook"
    }
  ],
  "created": 1662793907,
  "id": "cmpl-5oowVVZnAzdCPtUJ0rifeamtLcZRp",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 5,
    "total_tokens": 21
  }
}



### 일괄 처리 요청

OpenAI API에는 분당 요청 수와 분당 토큰 수에 대한 별도의 제한이 있습니다.

분당 요청 수 제한에 도달했지만 분당 토큰 수에 여유가 있는 경우, 여러 작업을 각 요청에 일괄 처리하여 처리량을 늘릴 수 있습니다. 이렇게 하면 특히 소규모 모델에서 분당 더 많은 토큰을 처리할 수 있습니다.

프롬프트 일괄 전송은 단일 문자열 대신 '프롬프트' 매개변수에 문자열 목록을 전달한다는 점을 제외하면 일반 API 호출과 완전히 동일하게 작동합니다.

**경고: 응답 객체가 프롬프트 순서대로 완료를 반환하지 않을 수 있으므로 항상 `index` 필드를 사용하여 응답을 프롬프트에 다시 일치시켜야 합니다.

#### 일괄 처리하지 않은 예제

In [5]:
import openai  # for making OpenAI API requests


num_stories = 10
prompt = "Once upon a time,"

# serial example, with one story completion per request
for _ in range(num_stories):
    response = openai.Completion.create(
        model="curie",
        prompt=prompt,
        max_tokens=20,
    )

    # print story
    print(prompt + response.choices[0].text)


Once upon a time, before there were grandiloquent tales of the massacre at Fort Mims, there were stories of
Once upon a time, a full-sized search and rescue was created. However, CIDIs are the addition of requiring
Once upon a time, Schubert was hot with the films. “Schubert sings of honey, flowers,
Once upon a time, you could watch these films on your VCR, sometimes years after their initial theatrical release, and there
Once upon a time, there was a forest. In that forest, the forest animals ruled. The forest animals had their homes
Once upon a time, there were two programs that complained about false positive scans. Peacock and Midnight Manager alike, only
Once upon a time, a long, long time ago, tragedy struck. it was the darkest of nights, and there was
Once upon a time, when Adam was a perfect little gentleman, he was presented at Court as a guarantee of good character.
Once upon a time, Adam and Eve made a mistake. They ate the fruit from the tree of immortality and split the co

#### 일괄 처리 예시

In [6]:
import openai  # for making OpenAI API requests


num_stories = 10
prompts = ["Once upon a time,"] * num_stories

# batched example, with 10 stories completions per request
response = openai.Completion.create(
    model="curie",
    prompt=prompts,
    max_tokens=20,
)

# match completions to prompts by index
stories = [""] * len(prompts)
for choice in response.choices:
    stories[choice.index] = prompts[choice.index] + choice.text

# print stories
for story in stories:
    print(story)


Once upon a time, there were two sisters, Eliza Pickering and Ariana 'Ari' Lucas. When these lovely
Once upon a time, Keene was stung by a worm — actually, probably a python — snaking through his leg
Once upon a time, there was a professor of physics during the depression. It was difficult, during this time, to get
Once upon a time, before you got sick, you told stories to all and sundry, and your listeners believed in you
Once upon a time, there was one very old nice donkey. He was incredibly smart, in a very old, kind of
Once upon a time, the property of a common lodging house was a common cup for all the inhabitants. Betimes a constant
Once upon a time, in an unspecified country, there was a witch who had an illegal product. It was highly effective,
Once upon a time, a long time ago, I turned 13, my beautiful dog Duncan swept me up into his jaws like
Once upon a time, as a thoroughly reformed creature from an army of Nazis, he took On Judgement Day myself and his
Once upon a time, C

## 병렬 처리 스크립트 예제

대량의 API 요청을 병렬 처리하기 위한 예제 스크립트를 작성했습니다: [api_request_parallel_processor.py](https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py).

이 스크립트에는 몇 가지 편리한 기능이 결합되어 있습니다:
- 대용량 작업으로 인한 메모리 부족을 방지하기 위해 파일에서 요청을 스트리밍합니다.
- 처리량을 최대화하기 위해 요청을 동시에 수행합니다.
- 요청과 토큰 사용량을 모두 스로틀하여 속도 제한을 준수합니다.
- 데이터 누락을 방지하기 위해 실패한 요청을 재시도합니다.
- 오류를 기록하여 요청 문제를 진단합니다.

있는 그대로 사용하거나 필요에 맞게 자유롭게 수정하세요.